In [1]:
from azureml.core import Workspace, Experiment, Run
import math, random, pickle, json
from azureml.core.model import Model
from azureml.core import Image
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice, Webservice, AksWebservice
import myImageLibrary
import cv2
import numpy as np
import os

##### 1 Setting the workspace from the config.json file and authenticating

In [4]:
ws = Workspace.from_config()

Found the config file in: C:\Projects\MachineLearning\dataminds-session\dataminds-session\config.json


##### 2 Registering the model

first fetch the model from a succesful previous run and save it in the working directory.

In [5]:
ws.models #list existing models in the workspace

<bound method Workspace.models of <azureml.core.workspace.Workspace object at 0x0000012443782D30>>

In [6]:
model = Model.register(ws, "my_model.h5","facemodel", description = "New facemodel, migrated to new Azure ML")

Registering model facemodel


You should now see the model in your Workspace in the Azure Portal. If you uploaded a model with this name before, it will get an incremented version tag. 

##### 3 Creating an image with the model, dependencies and scoring script

In [33]:
image_config = ContainerImage.image_configuration(execution_script = "newAzureML_score.py", 
                                    runtime = "python", 
                                    conda_file = "aml_config/newAzureML_env.yml")



Note: if you update the files and run the cell below again with the same image name, the new image will appear in the portal in the Workspace with a different version tag.

In [8]:
image = Image.create(ws,"faceimage",[model],image_config)

image.wait_for_creation(show_output = True)

Creating image
Running...........................................................
SucceededImage creation operation finished for image faceimage:10, operation "Succeeded"


image.wait_for_creation() is required here. Otherwise, if you proceed below to deploy the webservice from the image before it is fully created, it will raise an error saying that the image's creation state is not "Succeeded". The state will still be "running".

If you don't wait_for_creation(), the image will eventually appear as "Succeeded" in the Azure portal, but its state will not be updated here (image.creation_state will still return "running"). You must then manually update it with image.update_creation_state().

https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.image(class)?view=azure-ml-py#update-creation-state

In [9]:
image.creation_state # 'succeeded'

'Succeeded'

The image is now visible in your Workspace in the Azure portal


##### 4 Deploying a service to Azure Container Instances (ACI)


In [34]:
# aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1) # 
# from experience, the deployment will fail with this configuration. We need a more powerful deployment configuration.

aci_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 2)

In [17]:
service = Webservice.deploy_from_image(ws, "faceservice",image,
                                      deployment_config = aci_config)

service.wait_for_deployment(show_output = True)

Creating service
Running..................................................
SucceededACI service creation operation finished, operation "Succeeded"


##### 5 Inference

Loading an image, extracting the face, preprocessing and base64 encoding it, sending it to the service.

code taken from callservice.py

In [26]:
FACE_CASCADE = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [36]:
img_raw = cv2.imread("test_images/buscemi_test.jpg")
faces = myImageLibrary.extract_face(img_raw,FACE_CASCADE)
if len(faces) == 0:
    print("No face found. Try a different image")
else:
    face = myImageLibrary.resize_crop(faces[0],96)
    face = np.around(face/255.0, decimals = 12)    
    img_base64 = myImageLibrary.preprocess(face)
    person = service.run(img_base64)
    print(person)

{"index": "0"}


Succes! When running the example with buscemi_test.jpg, the index should be 0. 

##### 5 Deploying a service to AKS (TO DO)

In [ ]:
from azureml.core import ComputerTarget

In [ ]:
# configuring the AKS deployment
aks_config = AksWebservicevice.deploy_configuration()

In [ ]:
#configuring and provisioning a compute target. This was not required when deploying to ACI. 
prov_config = AksCompute.provisioning_configuration()
aks_target = ComputeTarget.create(ws, "first-aks-target",provisioning_configuration=prov_config)

# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.computetarget?view=azure-ml-py#create
# https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-to-aks

aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

In [ ]:
service_aks = Webservice.deploy_from_image(ws, "faceservice-aks",image,
                                      deployment_config = aks_config, deployment_target = aks_target)

service_aks.wait_for_deployment(show_output = True)

In [ ]:
service.run(img_base64)